In [1]:
from utils.data import Lemmatizer
from utils.loaders import IMDBDataset
import logging
import json

In [2]:
import numpy as np
import torch

In [3]:
import datasets
import transformers
from transformers import BertTokenizerFast, BertModel
from tqdm.auto import trange, tqdm

In [4]:
from utils.bert import get_embeddings_from_sentence, get_embeddings_from_document

In [5]:
from utils.clustering import BertHierarchicalClustering, BertCluster

In [6]:
imdb_dataset = IMDBDataset.load()
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
lemmatizer = Lemmatizer("en")
termins = json.load(open("termins.json"))

In [7]:
N = 20
hierarchical_clustering = BertHierarchicalClustering()

for i in tqdm(list(range(0, 10)) + list(range(12600, 12610))):
    embeddings = get_embeddings_from_document(imdb_dataset["train"][i]["text"], model, tokenizer, lemmatizer, termins)
    label = 0 if i < 12500 else 1
    cluster = BertCluster(embeddings, i, label)
    hierarchical_clustering.add_cluster(cluster)

  0%|          | 0/20 [00:00<?, ?it/s]

In [8]:
hierarchical_clustering.make_clusters(2)

  0%|          | 0/18 [00:00<?, ?it/s]

In [9]:
cluster_1, cluster_2 = hierarchical_clustering.clusters.values()

In [10]:
print(cluster_1.labels)
print(cluster_2.labels)

[1]
[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
